In [1]:
import altair as alt
import pandas as pd
alt.renderers.enable('mimetype')
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [2]:
pop = pd.read_csv('data/raw/API_SP.POP.TOTL_DS2_en_csv_v2_1678576.csv', skiprows=4).drop(
    columns=['Country Code', 'Indicator Name', 'Indicator Code', '2020', 'Unnamed: 65']).melt(id_vars=['Country Name'],
                                                                                              var_name='Year',
                                                                                              value_name='Population')
pop = pop.rename(columns={'Country Name': 'Country'})
pop["Country"] = pop["Country"].replace("West Bank and Gaza", 'Palestine')
pop

,Country,Year,Population
0,Aruba,1960,54211.0
1,Afghanistan,1960,8996973.0
2,Angola,1960,5454933.0
3,Albania,1960,1608800.0
4,Andorra,1960,13411.0
...,...,...,...
15835,Kosovo,2019,1794248.0
15836,"Yemen, Rep.",2019,29161922.0
15837,South Africa,2019,58558270.0
15838,Zambia,2019,17861030.0


In [3]:
crops = pd.read_csv('data/raw/Production_Crops_E_Asia_NOFLAG.csv')

In [4]:
paddy_wheat = crops.loc[(crops['Item Code'].isin(['15', '27', '30'])) & (crops['Element Code'] == 5510)].drop(
    columns=['Element Code', 'Element', 'Unit', 'Item Code', 'Area Code'])

In [5]:
paddy_wheat_melt = paddy_wheat.melt(id_vars=['Area', 'Item'],
                                    var_name='Year',
                                    value_name='Tonnes of crop').dropna()
paddy_wheat_melt['Year'] = paddy_wheat_melt['Year'].str.replace('Y','')
replace_countries = {'Viet Nam': 'Vietnam',
                     'Yemen': 'Yemen, Rep.',
                     'China, Hong Kong SAR': 'China',
                     'China, Taiwan Province of': 'China',
                     'China, mainland': 'China',
                     'Republic of Korea': 'Korea, Rep.',
                     'Democratic People\'s Republic of Korea': 'Korea, Dem. People’s Rep.',
                     'Iran (Islamic Republic of)': 'Iran, Islamic Rep.',
                     'Kyrgyzstan': 'Kyrgyz Republic',
                     'Lao People\'s Democratic Republic': 'Lao PDR'}
paddy_wheat_melt = paddy_wheat_melt.replace({"Area": replace_countries})

In [6]:
paddy_wheat_table = paddy_wheat_melt.pivot_table(values=['Tonnes of crop'], index=['Area', 'Year'], columns=['Item']).reset_index()
paddy_wheat_table.columns = ['Country', 'Year', 'Paddy_1', 'Paddy_2', 'Wheat']
paddy_wheat_table['Paddy'] = paddy_wheat_table['Paddy_1'] + paddy_wheat_table['Paddy_2']
paddy_wheat_table = paddy_wheat_table.drop(columns = ['Paddy_1', 'Paddy_2'])

In [7]:
final_df = pd.merge(paddy_wheat_table, pop, how="left", on=["Country", "Year"])
final_df

,Country,Year,Wheat,Paddy,Population
0,Afghanistan,1961,2279000.0,531773.0,9169410.0
1,Afghanistan,1962,2279000.0,531773.0,9351441.0
2,Afghanistan,1963,1947000.0,531773.0,9543205.0
3,Afghanistan,1964,2230000.0,633460.0,9744781.0
4,Afghanistan,1965,2282000.0,633460.0,9956320.0
...,...,...,...,...,...
2264,"Yemen, Rep.",2014,192215.0,NaN,25823485.0
2265,"Yemen, Rep.",2015,124940.0,NaN,26497889.0
2266,"Yemen, Rep.",2016,95917.0,NaN,27168210.0
2267,"Yemen, Rep.",2017,95651.0,NaN,27834821.0
